In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import random
import os

In [3]:
print(os.getcwd())
os.chdir('c:\\Users\\Gyanprakash\\Desktop\\Data-Science\\NLP')
print(os.getcwd())

c:\Users\Gyanprakash\Desktop\Data-Science\NLP\chatBot
c:\Users\Gyanprakash\Desktop\Data-Science\NLP


In [6]:
df = pd.read_csv('LSTM/fake_news_classifier/train.csv')

In [8]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [9]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
df = df.dropna()
df.reset_index(inplace=True)

In [11]:
df.isnull().sum()

index     0
id        0
title     0
author    0
text      0
label     0
dtype: int64

In [12]:
df.shape

(18285, 6)

In [13]:
df = df.iloc[:5000]

In [14]:
df.shape

(5000, 6)

In [15]:
X = df['title']

In [16]:
X

0       House Dem Aide: We Didn’t Even See Comey’s Let...
1       FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                       Why the Truth Might Get You Fired
3       15 Civilians Killed In Single US Airstrike Hav...
4       Iranian woman jailed for fictional unpublished...
                              ...                        
4995    The Arcturian Group by Marilyn Raffaele Octobe...
4996    Comment on Rachel Maddow declares that a Trump...
4997    HILLARY CLINTON is being funded by the same Mu...
4998                        Предикторские свойства толпы?
4999    Even If You Hate Trump, What this Leftist Just...
Name: title, Length: 5000, dtype: object

In [17]:
def random_values(feature_name:str,end=6):
    for i in range(0,end):
        # print(df[feature_name].sample(end))
        indx = np.random.randint(df.shape[0])
        print(f'{indx}: {df[feature_name].iloc[indx]}')

In [18]:
df['title'].sample(4).to_list()

['Ted Cruz Draws Democrat Challenger for 2018 Race',
 'Carlson: Trump-Russia ’Hoax’ an Effort to Topple a Democratically Elected Government the D.C. Permanent Class Doesn’t Like - Breitbart',
 'France Expels Swiss Islamist Preacher',
 'The Key To Healthy Bones And Teeth: Magnesium, NOT Calcium']

In [19]:
y = df['label']

In [20]:
y.value_counts()

label
0    2836
1    2164
Name: count, dtype: int64

In [21]:
## tensorflow
from keras.layers import (Embedding,
                          LSTM,
                          Dense)

from keras.utils import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot #This function receives as input a string of text
                                            #and returns a list of encoded integers each corresponding to a word (or token) in the given input string.
                                            # I will use for tokens


In [22]:
voc_size = 5000

In [23]:
## NLTK
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tqdm import tqdm

In [25]:
ps = PorterStemmer()
ps

<PorterStemmer>

In [26]:
random_values('text')

2995: And these people claim that Donald Trump is a racist! There goes Hillary’s black vote! Whenever the term “Nazi” is used to describe a minority group, you know things are bad. Democrat donor and Hillary Clinton supporter Benjamin Barber was caught on hidden camera voicing his opinion on blacks who may vote Republican. The disturbing comments were recorded at a fundraiser for United States Senate Candidate Deborah Ross, of North Carolina. Barber said, “Have you hear of the Sonderkommandos? Jewish guards who helped murder Jews in the camps. So there were even Jews that were helping the Nazis murder Jews! So black who are helping the other side are seriously f*cked in the head. They’re only helping the enemy who will destroy them. Maybe they think ‘if I help them we’ll get along okay; somehow I’ll save my race by working with the murderers.” Barber was caught in this disgusting tirade on the Upper West Side of New York City after the fundraiser in September. Watch for yourself: 
Besi

## Label

### 1: unreliable
#### 0: reliable

In [27]:
## Preprocessing
corpus = []
for i in tqdm(range(0,len(X))):
    txt = re.sub('[^a-zA-Z]',' ',X[i])
    txt = txt.lower()
    txt = txt.split()

    txt = [ps.stem(word) for word in txt if not word in stopwords.words('english')]
    txt = ' '.join(txt)
    corpus.append(txt)

  0%|          | 0/5000 [00:00<?, ?it/s]

100%|██████████| 5000/5000 [00:12<00:00, 385.48it/s]


In [28]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [29]:
sent_repr = [one_hot(input_text=sentences,n=voc_size) for sentences in corpus]
sent_repr

[[2490, 3332, 2147, 4200, 3969, 1150, 1817, 2552, 2679, 2614],
 [1622, 401, 617, 3880, 2371, 2114, 313],
 [1237, 3788, 2874, 4063],
 [1312, 3036, 3727, 1767, 9, 3446],
 [221, 2371, 2869, 2523, 3739, 900, 2371, 2442, 465, 3886],
 [3854,
  1329,
  4680,
  2160,
  3974,
  2122,
  3500,
  46,
  281,
  4091,
  4641,
  3702,
  4851,
  1345,
  313],
 [2493, 4564, 132, 2484, 3737, 977, 2873, 2, 276, 1309, 1509],
 [2012, 1026, 1857, 4988, 410, 631, 2122, 473, 276, 1309, 1509],
 [1959, 1250, 3585, 1050, 843, 4238, 2487, 2491, 2122, 4281],
 [4804, 3884, 4960, 2460, 402, 3193, 3151, 3071],
 [3853, 3467, 1227, 4188, 3637, 21, 2441, 521, 3783, 4252, 2252],
 [1767, 4032, 3969, 4238, 2122, 410],
 [1429, 4506, 4247, 4820, 4917, 2448, 298, 4449, 498],
 [4022, 541, 2799, 2983, 3508, 3069, 3595, 276, 1309, 1509],
 [2700, 272, 1524, 2346, 795, 276, 1309, 1509],
 [872, 3696, 213, 4917, 2994, 2514, 4163, 4762, 1355, 1409],
 [8, 4851, 401],
 [3403, 3486, 2087, 3678, 2122, 1254, 1917, 313],
 [1374, 2705, 617, 

In [30]:
## making all the sentences of same lenght
embedded_docs = pad_sequences(sequences=sent_repr,padding='post',maxlen=20)

In [31]:
embedded_docs[0]

array([2490, 3332, 2147, 4200, 3969, 1150, 1817, 2552, 2679, 2614,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

## Model

In [32]:
from tensorflow import keras

sigmoid = keras.activations.sigmoid

In [34]:
model_features = 50
model = Sequential(name='LSTM_model')

model.add(Embedding(
    input_dim=voc_size , output_dim=model_features,input_length=20
))


In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding

# Define your parameters
voc_size = 5000
model_features = 50
input_length = 20  # Input sequence length

# Create the model
model = Sequential(name='LSTM_model')
model.add(Embedding(input_dim=voc_size, output_dim=model_features, input_length=input_length))

# Compile the model (you might need to add more layers and compile accordingly)
model.compile(optimizer='adam', loss='mse')


input_data = np.array([...])  # Replace with your input data of shape (batch_size, input_length)

# Get the embedding layer's output for the input data
embedding_outputs = model.predict(input_data)

# Reshape the outputs to match the input shape
reshaped_outputs = embedding_outputs.reshape(input_data.shape[0], input_length, model_features)

# Store the embedding outputs in a DataFrame
output_df = pd.DataFrame(reshaped_outputs[0])  # Assuming you want to store the outputs for the first example

# Save the DataFrame to a CSV file
output_df.to_csv('embedding_outputs.csv', index=False)


In [35]:
model.summary()

Model: "LSTM_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 50)            250000    
                                                                 
Total params: 250,000
Trainable params: 250,000
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [36]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding

# Define your parameters
voc_size = 5000
model_features = 50

# Create the model
model = Sequential(name='LSTM_model')
model.add(Embedding(input_dim=voc_size, output_dim=model_features, input_length=20))

# Compile the model (you might need to add more layers and compile accordingly)
model.compile(optimizer='adam', loss='mse')

# Generate sample input data
sample_input = np.random.randint(0, voc_size, size=(10, 20))  # Adjust the batch size as needed

# Predict using the model
embedding_outputs = model.predict(sample_input)

# Store the outputs in a DataFrame
output_df = pd.DataFrame(embedding_outputs.reshape(embedding_outputs.shape[0], -1))

# Save the DataFrame to a CSV file
output_df


1/1 [==============================] - 0s 233ms/step


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.023547,0.049748,0.029330,-0.048885,0.003693,0.026711,-0.028614,-0.040928,-0.005043,-0.013690,...,0.037536,0.000318,0.016978,-0.016667,0.019678,-0.013615,0.013502,0.004931,0.027338,0.021820
1,0.047837,-0.035147,-0.039564,-0.048138,0.023832,-0.047674,0.020982,0.004322,0.047544,-0.004202,...,-0.015314,0.007484,0.002779,0.016217,0.024277,0.015574,-0.046404,-0.017069,0.038541,0.043190
2,0.049987,0.043615,0.048225,-0.006413,-0.048965,-0.032903,0.017663,0.045379,0.004501,0.033462,...,-0.013546,-0.008312,0.045749,0.049464,0.022795,-0.007204,0.041476,-0.037782,0.038785,0.025936
3,-0.040306,-0.011192,-0.020592,0.037089,-0.008860,0.016608,-0.035826,0.036318,0.030404,-0.011616,...,-0.001725,-0.015141,-0.016102,-0.035441,0.045454,0.023210,-0.035243,-0.048786,0.017184,-0.019837
4,-0.019195,0.024074,-0.001464,-0.008834,0.000918,-0.037199,0.044970,0.028274,-0.035254,-0.009792,...,-0.029555,-0.007593,-0.034098,0.012324,0.045103,0.048907,-0.022290,-0.008104,0.019829,0.022404
5,-0.015277,0.032961,0.041654,-0.041313,0.024212,0.018032,-0.024268,-0.034326,-0.009429,0.043927,...,0.041564,-0.023372,0.018844,-0.037239,-0.028276,-0.033550,0.004407,-0.042237,-0.032977,-0.022705
6,-0.013503,-0.046227,0.001935,0.035159,-0.044724,-0.006457,-0.033493,-0.044349,-0.001893,-0.044570,...,-0.030257,0.032022,0.009606,-0.041957,0.032046,0.041825,-0.033629,0.019446,0.032570,0.042118
7,-0.003907,-0.045962,0.018351,-0.001672,-0.009411,0.042398,0.020499,0.004461,-0.004857,-0.037458,...,-0.048380,0.023518,-0.010436,-0.035536,-0.005431,0.025692,0.041013,-0.030951,0.015228,0.039991
8,0.029142,-0.044573,0.044247,-0.023276,-0.026542,0.009295,0.041646,0.003457,0.033984,0.012971,...,0.008505,-0.034367,-0.031105,0.023411,0.036465,0.045456,0.031752,-0.039356,0.005638,0.022675
9,0.043316,-0.026356,-0.003970,0.048293,-0.007766,-0.047266,-0.014117,-0.023928,-0.025332,-0.015161,...,-0.007342,0.042523,-0.026132,0.019294,0.038858,0.020937,0.030987,0.049685,0.034974,0.038748
